In [ ]:
import weave
import io
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import random
from weave.legacy.weave import ops

def plt_image():
    """Return current matplotlib figure as PIL Image"""
    img_buf = io.BytesIO()
    plt.savefig(img_buf, format='png')
    im = Image.open(img_buf)
    return im

In [ ]:
# Declare an op that plots a sin weave using matplotlib, and returns the plot as an Image

@weave.op(render_info={"type": "function"})
def sin_image(f: int) -> Image.Image:    
    x = np.arange(0, f * np.pi, 0.1)
    y = np.sin(x)
    plt.rcParams["figure.figsize"] = (22, 7)
    plt.plot(x, y)
    
    return plt_image()

In [ ]:
x = weave.save(7, name='my-number')

im = sin_image(x + 2)
im

In [ ]:
import json
from weave.legacy.weave.ops_domain import wb_domain_types
from weave import gql_op_plugin
import wandb


@weave.op(
    render_info={"type": "function"},
    plugins=gql_op_plugin.wb_gql_op_plugin(
        lambda inputs, inner: "project {id name entity {id name}}"
    )
)
def run_accuracy_barchart(runs: list[wb_domain_types.Run]) -> Image.Image:
    runs = list(runs)[:10]
    runs = [wandb.Api().run(f'{r["project"]["entity"]["name"]}/{r["project"]["name"]}/{r["name"]}') for r in runs]
    names = [r.name for r in runs]
    xs = range(len(runs))
    ys = [r.summary_metrics.get("acc") or 0 for r in runs]

    plt.rcParams["figure.figsize"] = (15, 5)
    plt.bar(xs, ys, align="center", alpha=0.5)
    plt.xticks(xs, names)
    plt.ylabel("acc")
    plt.title("Run accuracies")

    return plt_image()

In [ ]:
runs = weave.legacy.weave.ops.project('shawn', 'fasion-sweep').runs()
run_accuracy_barchart(runs)